In [76]:
import dowhy
from dowhy import CausalModel
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
from sys import path
import os
for dirname, _, filenames in os.walk('../resources/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
path.append(os.path.realpath('../'))

from custom import functions

warnings.filterwarnings('ignore')

../resources/allColProSol.xlsx
../resources/dataset_a_2021.csv
../resources/dataset_a_2021c2.csv
../resources/dataset_a_2021v1.xlsx
../resources/dataset_unab_ORIGINAL SIN P1.csv
../resources/dataset_unab_P1.csv
../resources/exitoFallidoEnviosProgramaSol.xlsx
../resources/exitoFallidoProgramaSol.xlsx
../resources/exitosoFallidosEnviosAllColProSol.xlsx
../resources/exitosoFallidosEnviosAllColSol.xlsx
../resources/exitososFallidosEnviosSol.xlsx
../resources/Hito12sinColCeroExitosoFallidosSol.xlsx
../resources/sinColCeroExitosoFallidosEnviosSol.xlsx
../resources/sinColCeroExitosoFallidosSol.xlsx
../resources/v2_hitosExitoFalloColESol1.csv
../resources/v2_hitosExitoFalloColESol1Prograna.csv
../resources/v2_hitosExitoFalloSol1Programa.csv
../resources/causalidad\causalidad.dot
../resources/causalidad\causalidad.png
../resources/causalidad\causalidad2.dot
../resources/causalidad\causalidad_e29.dot
../resources/causalidad\causalidad_exitosos.dot
../resources/causalidad\causalidad_hito1.dot
../

In [77]:
# Read and preview data
df = pd.read_csv("../resources/v2_hitosExitoFalloColESol1.csv", delimiter=";", skipinitialspace=True)

In [78]:
#creando columna aprobado y con la funcion set_in_aprobado_nota poblamos la nueva columna.
df['aprobado']=df.apply(lambda x: functions.set_in_aprobado_nota(x['sol1']),axis = 1 )
#revisamos la existencia de la nueva columna.
print(df.columns)

Index(['hito1', 'hito2', 'exitosos', 'fallidos', 'e0', 'e1', 'e2', 'e3', 'e4',
       'e5', 'e6', 'e7', 'e8', 'e9', 'e10', 'e11', 'e12', 'e13', 'e14', 'e15',
       'e16', 'e17', 'e18', 'e19', 'e20', 'e21', 'e22', 'e23', 'e24', 'e25',
       'e26', 'e27', 'e28', 'e29', 'e30', 'e31', 'e32', 'e33', 'e34', 'e35',
       'e36', 'e37', 'e38', 'e39', 'e40', 'e41', 'e42', 'e43', 'e44', 'e45',
       'e46', 'e47', 'e48', 'e49', 'e50', 'e51', 'e52', 'sol1', 'aprobado'],
      dtype='object')


In [79]:
#creando graph manual.
causal_graph = """
digraph {
    e42 -> aprobado;
    e35 -> aprobado;
    e29 -> aprobado;
    e3 -> aprobado;
}
"""

In [80]:
# Paso 1: Modelar un problema causal
model = CausalModel(
    data=df,
    graph=causal_graph,
    treatment='e29',  # Variable tratada (exposición)
    outcome='aprobado',  # Variable de resultado
    common_causes=['e42','e35','e3']  # Variables de causa común
)

In [81]:
# Visualizar el modelo causal
model.view_model()

In [82]:
# Paso 2: Identificar el estimando objetivo bajo el modelo
identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)

In [83]:
# Paso 3: Estimar el efecto causal con el estimador de categoría econml TLearner
estimate = model.estimate_effect(
    identified_estimand,
    method_name="backdoor.econml.metalearners.TLearner",
    confidence_intervals=False,
    method_params={
        "init_params": {"models": RandomForestRegressor()},
        "fit_params": {},
    },
)

In [84]:
# Paso 4: Refutar el estimado obtenido
refute_results = model.refute_estimate(identified_estimand, estimate, method_name="placebo_treatment_refuter", placebo_type="permute")

In [85]:
# Paso 5: Imprimir resultados
print("----------------------------")
print("Estimado de efecto causal:")
print(estimate.value)
print("----------------------------")

----------------------------
Estimado de efecto causal:
0.23012176047320804
----------------------------


In [86]:
print("----------------------------")
print("Refutación de estimado:")
print(refute_results)
print("----------------------------")

----------------------------
Refutación de estimado:
Refute: Use a Placebo Treatment
Estimated effect:0.23012176047320804
New effect:-0.001631137641560498
p value:1.0

----------------------------


In [87]:
print("----------------------------")
print("¿Es aplicable la puerta trasera?")
print(estimate)
print("----------------------------")

----------------------------
¿Es aplicable la puerta trasera?
*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
  d                
──────(E[aprobado])
d[e₂₉]             
Estimand assumption 1, Unconfoundedness: If U→{e29} and U→aprobado then P(aprobado|e29,,U) = P(aprobado|e29,)

## Realized estimand
b: aprobado~e29+e42+e35+e3
Target units: ate

## Estimate
Mean value: 0.23012176047320804
Effect estimates: [ 0.28826171  0.23135516  0.23135516  0.23135516  0.23135516  0.23135516
  0.23135516  0.23135516  0.23135516  0.23135516  0.23135516  0.23135516
  0.23135516  0.23135516  0.23135516  0.13281011  0.23135516  0.22475606
  0.23135516  0.23135516  0.23135516  0.23135516  0.23135516  0.23135516
  0.23135516  0.23135516  0.23135516  0.23135516  0.23135516  0.23135516
  0.23135516  0.39855762  0.23135516  0.23135516  0.23135516  0.23135516
  0.23135516  0.23135516  0.23135516  0.23135516  0.2313